In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report
import pandas as pd


In [3]:
enc = LabelEncoder()

sports_tweets = pd.read_csv("POOP.csv")

# Encode the Sport feature using a new column labeled "Sport_Cat"
sports_tweets["Sport_Cat"] = enc.fit_transform(sports_tweets["Sport"])

In [4]:
x = sports_tweets["Tweet"]
y = sports_tweets["Sport_Cat"]

# Create the Vectorizer that uses both unigrams and bigrams with tuple (1, 2)
tfidf = TfidfVectorizer(max_features=1000, ngram_range=(1, 2))

In [5]:
sports_tweets

Tweet              Sport  \
0    extremely rare footage of the elusive box jell...             Soccer   
1        behind the scenes of the best nba commercial.         Basketball   
2                         protect the football. simple  American football   
3                           elite level of basketball.         Basketball   
4                                                  NaN  American football   
..                                                 ...                ...   
840   nelson county vs. columbia stream now we pres...         Basketball   
841  top 3 in football history 1 messi 2 maradona 3...             Soccer   
842  has any owner in nba history done less with mo...         Basketball   
843  extremely rare footage of the elusive box jell...             Soccer   
844  lol just your loss and move on man. football p...  American football   

     Sport_Cat  
0           12  
1            3  
2            0  
3            3  
4            0  
..         ...  
840          3  
841         12  
842          3  
843         12  
844          0  

[845 rows x 3 columns]

In [6]:
# Change X to include those unigrams and bigrams
x_fit = tfidf.fit_transform(sports_tweets['Tweet'].values.astype(str))

# Split the data
# x_train, x_test, y_train, y_test = train_test_split(x_fit, y, test_size=0.2)
skfold = StratifiedKFold(n_splits = 10)

In [7]:
lin_svc = LinearSVC()
rbf_svc = SVC()
knn = KNeighborsClassifier()
log_reg = LogisticRegression()
rfc = RandomForestClassifier(n_estimators=50)
mlp = MLPClassifier()

In [8]:
def average(arr) -> float:
    # Returns the average of the array, arr, as a float
    return sum(arr) / len(arr)

In [11]:
# Test the data
lin_svc_scores = cross_val_score(lin_svc, x_fit, y, cv=skfold)
rbf_svc_scores = cross_val_score(rbf_svc, x_fit, y, cv=skfold)
knn_scores = cross_val_score(knn, x_fit, y, cv=skfold)
log_reg_scores = cross_val_score(log_reg, x_fit, y, cv=skfold)
rfc_scores = cross_val_score(rfc, x_fit, y, cv=skfold)
mlp_scores = cross_val_score(mlp, x_fit, y, cv=skfold)


C:\Users\jazli\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\jazli\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\jazli\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\jazli\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\jazli\AppData\Roaming\Python\Python311\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y

In [12]:
def choose_classifier(lin_svc, rbf_svc, knn, log_reg, rfc, mlp):
    empty_dict = {}
    empty_dict["lin_svc"] = average(lin_svc_scores)
    empty_dict["rbf_svc"] = average(rbf_svc_scores)
    empty_dict["knn"] = average(knn_scores)
    empty_dict["log"] = average(log_reg_scores)
    empty_dict["rfc"] = average(rfc_scores)
    empty_dict["mlp"] = average(mlp_scores)

    return string_to_classifier(max(empty_dict, key=empty_dict.get))

def string_to_classifier(string):
    match string:
        case "lin_svc":
            return lin_svc
        case "rbf_svc":
            return rbf_svc
        case "knn":
            return knn
        case "log":
            return log_reg
        case "rfc":
            return rfc
        case "mlp":
            return mlp

# Choose correct classifier
classifier = choose_classifier(lin_svc, rbf_svc, knn, log_reg, rfc, mlp)

In [14]:
print(average(lin_svc_scores))
print(average(rbf_svc_scores))
print(average(knn_scores))
print(average(log_reg_scores))
print(average(rfc_scores))
print(average(mlp_scores))


0.8496918767507002
0.8284033613445377
0.5503641456582632
0.8225210084033613
0.8509383753501399
0.8118627450980392


In [170]:
# Once classifier is chosen then split the data into training and testing

x_train, x_test, y_train, y_test = train_test_split(x_fit, y, test_size=0.2)

In [171]:
x = classifier.fit(x_train, y_train)
x.score(x_test, y_test)

y_pred = classifier.predict(x_test)


In [172]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.83      0.84        35
           3       0.95      0.98      0.96        84
           5       0.00      0.00      0.00         1
           6       1.00      1.00      1.00         1
           8       1.00      0.80      0.89        10
          11       0.00      0.00      0.00         1
          12       0.80      0.89      0.84        36
          14       0.00      0.00      0.00         1

    accuracy                           0.90       169
   macro avg       0.58      0.56      0.57       169
weighted avg       0.89      0.90      0.89       169



C:\Users\jazli\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jazli\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\jazli\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)